In [ ]:
import struct
import numpy

In [ ]:
#Put all bytes from target file into memmap array
target_file = numpy.memmap(filename = "tests/samples/dummy.root", mode = "r", dtype = numpy.uint8, shape = (928,))

In [ ]:
_format1       = ">4si"
_format2_small = ">iiiiiiBiii18s"
_format2_big   = ">iqqiiiBiqi18s"
_format3       = ">hIIii"
_format4_small = ">iii"
_format4_big   = ">qqq"
_format5       = ">i"
_format_small  = ">ihiIhhii"
_format_big    = ">ihiIhhqq"

In [ ]:
name = input("Enter name of the ROOT file - ")
if name.endswith(".root") is False:
    name = name + ".root"
file = numpy.memmap(filename = name, dtype = numpy.uint8, mode = "w+", shape = (928,))
#In the case of a 1D array, the shape is the size of the file
#In ROOT files, the fEND corresponds to the size of the file
#fEND for ~/Downloads/uproot/tests/samples/dummy.root is 928

In [ ]:
#Working on just replacing bytes on target_file
file = numpy.memmap(filename = "rootpy.root", dtype = numpy.uint8, mode = "w+", shape = (928,))
file[:] = target_file[:]

In [ ]:
#Header bytes
fVersion = 61400
toadd = numpy.frombuffer(struct.pack(_format1, b"root", fVersion), dtype=numpy.uint8)
count = len(toadd)
file[:count] = toadd
#constants for file dummy.root
fBEGIN = 100
fEND = 822
fSeekFree = 874
fNbytesFree = 54
nfree = 1
fNbytesName = 56
fUnits = 4
fCompress = 0
fSeekInfo = 318
fNbytesInfo = 434
fUUID = b"\x00\x010\xd5\xf5\xea~\x0b\x11\xe8\xa2D~S\x1f\xac\xbe\xef"
toadd = numpy.frombuffer(struct.pack(_format2_small, fBEGIN, fEND, fSeekFree, fNbytesFree, nfree, fNbytesName, fUnits, fCompress, fSeekInfo, fNbytesInfo, fUUID), dtype=numpy.uint8)
file[count:(count + len(toadd))] = toadd
count = count + len(toadd)

In [ ]:
#Tkey instance
start = 100
fNbytes = 116
fVersion = 4
fObjlen = 72
fDatime = 1573188772
fKeylen = 44
fCycle = 1
fSeekKey = 100
fSeekPdir = 0
toadd = numpy.frombuffer(struct.pack(_format_small, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir), dtype = numpy.uint8)
file[start:(start + len(toadd))] = toadd
newpos = 126
fClassName = b'TFile'
fName = b'dummy.root'
fTitle = b''
file[newpos] = bytes(str(len(fClassName)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fClassName))] = numpy.frombuffer(fClassName, dtype = numpy.uint8)
newpos = newpos + len(fClassName)
file[newpos] = bytes(str(len(fName)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fName))] = numpy.frombuffer(fName, dtype = numpy.uint8)
newpos = newpos + len(fName)
file[newpos] = bytes(str(len(fTitle)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fTitle))] = numpy.frombuffer(fTitle, dtype = numpy.uint8)

In [ ]:
#Value of TKey
newpos = 156
fVersion = 5
fDatimeC = 1573188772
fDatimeM = 1573188772
fNbytesKeys = 122
fNbytesName = 56
fSeekDir = 100
fSeekParent = 0
fSeekKeys = 752
toadd = numpy.frombuffer(struct.pack(_format3, fVersion, fDatimeC, fDatimeM, fNbytesKeys, fNbytesName), dtype = numpy.uint8)
file[newpos:(newpos + len(toadd))] = toadd
newpos = newpos + len(toadd)
toadd = numpy.frombuffer(struct.pack(_format4_small, fSeekDir, fSeekParent, fSeekKeys), dtype = numpy.uint8)
file[newpos:(newpos + len(toadd))] = toadd

In [ ]:
#Read number of keys
nkeys = 1
newpos = 796
toadd = numpy.frombuffer(struct.pack(_format5, nkeys), dtype = numpy.uint8)
file[newpos:(newpos + len(toadd))] = toadd

In [ ]:
#Tkey instance
start = 752
fNbytes = 122
fVersion = 4
fObjlen = 48
fDatime = 1573188772
fKeylen = 44
fCycle = 1
fSeekKey = 752
fSeekPdir = 100
toadd = numpy.frombuffer(struct.pack(_format_small, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir), dtype = numpy.uint8)
file[start:(start + len(toadd))] = toadd
newpos = 778
fClassName = b'TFile'
fName = b'dummy.root'
fTitle = b''
file[newpos] = bytes(str(len(fClassName)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fClassName))] = numpy.frombuffer(fClassName, dtype = numpy.uint8)
newpos = newpos + len(fClassName)
file[newpos] = bytes(str(len(fName)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fName))] = numpy.frombuffer(fName, dtype = numpy.uint8)
newpos = newpos + len(fName)
file[newpos] = bytes(str(len(fTitle)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fTitle))] = numpy.frombuffer(fTitle, dtype = numpy.uint8)

In [ ]:
#Tkey instance 
start = 800
fNbytes = 102
fVersion = 4
fObjlen = 28
fDatime = 1573188772
fKeylen = 74
fCycle = 1
fSeekKey = 216
fSeekPdir = 100
toadd = numpy.frombuffer(struct.pack(_format_small, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir), dtype = numpy.uint8)
file[start:(start + len(toadd))] = toadd
newpos = 826
fClassName = b'TObjString'
fName = b'Hello World'
fTitle = b'Collectable string class'
file[newpos] = bytes(str(len(fClassName)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fClassName))] = numpy.frombuffer(fClassName, dtype = numpy.uint8)
newpos = newpos + len(fClassName)
file[newpos] = bytes(str(len(fName)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fName))] = numpy.frombuffer(fName, dtype = numpy.uint8)
newpos = newpos + len(fName)
file[newpos] = bytes(str(len(fTitle)),"ascii")
newpos = newpos + 1
file[newpos:(newpos + len(fTitle))] = numpy.frombuffer(fTitle, dtype = numpy.uint8)

In [ ]:
#Tkey streamer
start = 344
fNbytes = 434
fVersion = 4
fObjlen = 370
fDatime = 1573188772
fKeylen = 64
fCycle = 1
fSeekKey = 318
fSeekPdir = 100
toadd = numpy.frombuffer(struct.pack(_format_big, fNbytes, fVersion, fObjlen, fDatime, fKeylen, fCycle, fSeekKey, fSeekPdir), dtype = numpy.uint8)
file[start:(start + len(toadd))] = toadd

#Garbage till now
file[344:377] = target_file[344:377]

In [ ]:
#Examine bytes of target file
f = open("tests/samples/dummy.root", "rb")
count = 0
try:
    byte = f.read(1)
    while byte != "":
        print (count, " ", byte)
        count = count + 1
        byte = f.read(1)
        if count == 928:
            break
finally:
    f.close()

In [ ]:
#Examine bytes of target file copy
f = open("tests/samples/dummy2.root", "rb")
count = 0
try:
    byte = f.read(1)
    while byte != "":
        print (count, " ", byte)
        count = count + 1
        byte = f.read(1)
        if count == 928:
            break
finally:
    f.close()

In [ ]:
#Examine bytes of our file
f = open("rootpy.root", "rb")
count = 0
try:
    byte = f.read(1)
    while byte != "":
        print (count, " ", byte)
        count = count + 1
        byte = f.read(1)
        if count == 928:
            break
finally:
    f.close()

In [ ]:
#Examine bytes of empty file
f = open("tests/samples/empty.root", "rb")
count = 0
try:
    byte = f.read(1)
    while byte != "":
        print (count, " ", byte)
        count = count + 1
        byte = f.read(1)
        if count == 403:
            break
finally:
    f.close()

In [ ]:
#Examining target_file
target_file[:count]

In [ ]:
#Examining our file
file[:count]

In [ ]:
#Test - Debug target file with uproot
import uproot
uproot.open("~/Downloads/uproot/tests/samples/dummy.root")

In [ ]:
#Test - Examine same file with TObjString
import uproot
uproot.open("~/Downloads/uproot/tests/samples/dummy2.root")

In [ ]:
#Test - Check hexdump of target file
!hexdump -C ~/Downloads/uproot/tests/samples/dummy.root

In [ ]:
#Test - Hexdump of basic file
!hexdump -C ~/Downloads/uproot/tests/samples/dummy2.root

In [ ]:
#Test - See bytes read of copied file
!hexdump -C rootpy.root | head

In [ ]:
#Test - uproot is reading is reading our file correctly
import uproot
uproot.open("~/Downloads/uproot/rootpy.root", read_streamers = False)

In [ ]:
#Test - Check size of file
import os
statinfo_target = os.stat('tests/samples/sample-6.14.00-uncompressed.root')
print ("Size of target file is ", statinfo_target.st_size)
statinfo_copy = os.stat('rootpy.root')
print ("Size of written file is ", statinfo_copy.st_size)